In [9]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
import random

In [10]:
stocks= pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [11]:
from kamka import IEX_CLOUD_API_TOKEN

In [12]:
from json import JSONDecodeError

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

try:
    data = requests.get(api_url).json()
    print(data)
except JSONDecodeError as e:
    print(f"Failed to decode JSON: {e}")



Failed to decode JSON: Expecting value: line 1 column 1 (char 0)


In [13]:
#data['marketCap']
#data['latestPrize']

In [14]:
my_cols= ['Ticker','Market Capital','Prize','Number of shares to buy']
final_dataframe= pd.DataFrame(columns = my_cols)
final_dataframe

,Ticker,Market Capital,Prize,Number of shares to buy


In [15]:
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    #data= requests.get(api_url).json()
    mcap = random.randint(1,5)
    prize= random.randint(100,400)
    #prize= data['latestPrice']
    # mcap= data['marketCap']
    
    final_dataframe.loc[len(final_dataframe)]=[symbol,mcap,prize,'NA']
final_dataframe

,Ticker,Market Capital,Prize,Number of shares to buy
0,A,4,353,NA
1,AAL,5,150,NA
2,AAP,4,202,NA
3,AAPL,4,400,NA
4,ABBV,4,151,NA
...,...,...,...,...
500,YUM,4,368,NA
501,ZBH,3,108,NA
502,ZBRA,2,296,NA
503,ZION,5,202,NA


In [7]:
moneyToInvest= float(input("Enter the money you want to invest."))
moneyPerStock= moneyToInvest/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(moneyPerStock / final_dataframe['Prize'][i])

final_dataframe

Enter the money you want to invest. 6009000


,Ticker,Market Capital,Prize,Number of shares to buy
0,A,4,236,50
1,AAL,5,292,40
2,AAP,2,236,50
3,AAPL,1,273,43
4,ABBV,2,318,37
...,...,...,...,...
500,YUM,5,397,29
501,ZBH,5,260,45
502,ZBRA,5,393,30
503,ZION,1,274,43


In [21]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Market Capital', dollar_format],
                    'C': ['Prize', dollar_format],
                    'D': ['Number of shares to buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)